In [8]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression, mutual_info_regression
from label_parsing import label_parser
import matplotlib.pyplot as plt

# Read in Data Set + Info

In [4]:
df = pd.read_csv('adult21.csv')
df.head()

URBRRL  RATCAT_A  IMPINCFLG_A  CVDVAC2YR_A  CVDVAC2MR_A  CVDVAC1YR_A  \
0       4         7            0          NaN          NaN          NaN   
1       4        12            0          NaN          NaN          NaN   
2       4        14            0          NaN          NaN          NaN   
3       3        11            0          NaN          NaN          NaN   
4       1         6            1          NaN          NaN          NaN   

   CVDVAC1MR_A  SHTCVD19AV_A  SHTCVD19NM_A  SHTCVD19_A  ...  PROXYREL_A  \
0          NaN           NaN           NaN         NaN  ...         NaN   
1          NaN           NaN           NaN         NaN  ...         NaN   
2          NaN           NaN           NaN         NaN  ...         NaN   
3          NaN           NaN           NaN         NaN  ...         NaN   
4          NaN           NaN           NaN         NaN  ...         NaN   

   PROXY_A  AVAIL_A  HHSTAT_A  INTV_MON  RECTYPE  IMPNUM_A     WTFA_A  \
0      NaN        1         1         1       10         1   5423.324   
1      NaN        1         1         1       10         1   3832.196   
2      NaN        1         1         1       10         1   3422.661   
3      NaN        1         1         1       10         1  12960.165   
4      NaN        1         1         1       10         1   9284.618   

       HHX  POVRATTC_A  
0  H056808        1.93  
1  H018779        4.45  
2  H049265        5.94  
3  H007699        3.70  
4  H066034        1.66  

[5 rows x 622 columns]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29482 entries, 0 to 29481
Columns: 622 entries, URBRRL to POVRATTC_A
dtypes: float64(437), int64(184), object(1)
memory usage: 139.9+ MB


# Cleaning

In [6]:
# Vars
countColNan = df.isnull().sum(axis=0).tolist() # Column NaN counts
numRows = len(df) # Number of rows
colNames = df.columns

percList = [((x / numRows)*100) for x in countColNan] # Percent Missing Data

addBool = [ind < 1  for ind in percList] 
addList = []

for i in range(len(addBool)):
    if addBool[i]:
        addList.append(colNames[i])



dfNew = df[addList].copy()
dfNew = dfNew.drop(['HHX'], axis=1)
dfNew.dropna()

print("Number of remaining Columns:", len(dfNew.columns))

goodCols = ['EDUCP_A', 'PHSTAT_A','LSATIS11R_A', 'LSATIS4R_A', 'ANXEV_A', 'DEPEV_A','WEIGHTLBTC_A','NOTCOV_A']#'WELLVIS_A']

dfPCA = dfNew[goodCols]

Number of remaining Columns: 189


In [15]:
df = pd.read_csv('adult21.csv')

# threshold for null values
null_thresh = 0.01

pruned_df = df[df.columns[df.isnull().mean() < null_thresh]]

print(len(pruned_df.columns))
# Set up the label parser
lp = label_parser()

190


# Dimension Reduction

In [35]:
targetDesc = ["Targeting by life satisfaction (chi2)","Targeting by weight (chi2)"]
targetList = ['LSATIS11R_A', 'WEIGHTLBTC_A']
dfList = ['df1','df2']


for index in range(len(targetList)):
    features = list(dfTest.columns)
    features.remove(targetList[index])

    topList = []
    y = dfTest[targetList[index]]
    X = dfTest[features]

    bestfeatures = SelectKBest(score_func=chi2, k=15)
    fit = bestfeatures.fit(X,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)

    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  
    topList = featureScores.nlargest(15,'Score')
    dfList[index] = pd.DataFrame(topList)

    nameList = []
    for i in dfList[index]["Specs"]:
        nameList.append(lp.find_var_label(i))

    dfList[index]["Description"] = nameList
    dfList[index]["Target"] = targetDesc[index]

In [26]:
dfList[0]

Specs         Score                                Description  \
20  WEIGHTLBTC_A  29439.183665  Weight without shoes (pounds), public use   
9     CITZNSTP_A  11008.884074                         Citizenship status   
61   NATUSBORN_A  10049.626626             Born in U.S. or U.S. territory   
15     FDSCAT3_A   9879.960904      Adult 3 category food security recode   
16  EMPWRKLSW1_A   9669.733605                           Worked last week   
14     FDSCAT4_A   9225.390292     Adult  4 category food security recode   
59     INCWRKO_A   8025.874761                          Income from wages   
75     TASTEDF_A   7980.873946                         Difficulty tasting   
52   HOUTENURE_A   7693.939514                     Residence owned/rented   
78     SMELLDF_A   7472.813458                        Difficulty smelling   
66     SUPPORT_A   7333.128132         How often social/emotional support   
71       SMKEV_A   6060.676701                 Ever smoked 100 cigarettes   
69     CIGAREV_A   5758.275797                        Ever smoked a cigar   
62       AFVET_A   5263.848994            Ever serve active duty military   
58    INCINTER_A   5218.641198                       Income from accounts   

                                   Target  
20  Targeting by life satisfaction (chi2)  
9   Targeting by life satisfaction (chi2)  
61  Targeting by life satisfaction (chi2)  
15  Targeting by life satisfaction (chi2)  
16  Targeting by life satisfaction (chi2)  
14  Targeting by life satisfaction (chi2)  
59  Targeting by life satisfaction (chi2)  
75  Targeting by life satisfaction (chi2)  
52  Targeting by life satisfaction (chi2)  
78  Targeting by life satisfaction (chi2)  
66  Targeting by life satisfaction (chi2)  
71  Targeting by life satisfaction (chi2)  
69  Targeting by life satisfaction (chi2)  
62  Targeting by life satisfaction (chi2)  
58  Targeting by life satisfaction (chi2)

In [27]:
dfList[1]

Specs         Score  \
20     HEIGHTTC_A  24261.587400   
19       BMICAT_A  10956.425163   
3            PPSU   6217.966962   
50         AGEP_A   5460.670727   
26    LSATIS11R_A   4357.351769   
36        EDUCP_A   1779.558553   
49          SEX_A   1000.759805   
22   EMERG12MTC_A    895.627899   
1        RATCAT_A    845.284013   
177      PHSTAT_A    735.434179   
11      MARSTAT_A    710.640931   
2     IMPINCFLG_A    705.801126   
43       NUMCAN_A    582.522358   
105    CHOLLAST_A    574.176293   
21   URGNT12MTC_A    566.502227   

                                           Description  \
20           Height without shoes (inches), public use   
19             Categorical Body Mass Index, public use   
3    Pseudo-PSU for public-use file variance estima...   
50                               Age of SA (top coded)   
26            Life satisfaction - 11 category question   
36                   Educational level of sample adult   
49                                 Sex of Sample Adult   
22   Number of times visited hospital emergency roo...   
1    Ratio of family income to poverty threshold fo...   
177                              General health status   
11              Current marital status of sample adult   
2             Imputed SA family income imputation flag   
43                 Number of reported types of cancers   
105                      Last time cholesterol checked   
21   Number of times visited urgent care, past 12m,...   

                         Target  
20   Targeting by weight (chi2)  
19   Targeting by weight (chi2)  
3    Targeting by weight (chi2)  
50   Targeting by weight (chi2)  
26   Targeting by weight (chi2)  
36   Targeting by weight (chi2)  
49   Targeting by weight (chi2)  
22   Targeting by weight (chi2)  
1    Targeting by weight (chi2)  
177  Targeting by weight (chi2)  
11   Targeting by weight (chi2)  
2    Targeting by weight (chi2)  
43   Targeting by weight (chi2)  
105  Targeting by weight (chi2)  
21   Targeting by weight (chi2)